In [ ]:
import numpy as np
import scipy as sp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees

In [3]:
def move_score(G,H, mapping, u, x):

    f_u = mapping[u]

    u_row = G[u,:]
    u_col = G[:,u]
    f_u_row = H[[f_u],mapping]
    f_u_col = H[mapping,[f_u]]
    x_row = H[[x],mapping]
    x_col = H[mapping,[x]]

    old_score = np.minimum(u_row,f_u_row).sum() + np.minimum(u_col,f_u_col).sum()
    old_score = old_score - min(G[u,u],H[f_u,f_u])
    new_score = np.minimum(u_row,x_row).sum() + np.minimum(u_col,x_col).sum()
    new_score = new_score - min(G[u,u],H[x,x])

    return new_score - old_score

def find_best_move(G,H, mapping, u, mask = None):

    m = G.shape[0]
    u_row = G[u,:]
    u_col = G[:,u]

    if mask is None:
        n = H.shape[0]
        X = H[:,mapping[None,:]].reshape((n,m))
        Y = H[mapping[:,None],:].reshape((m,n))
    else:
        X = H[mask,mapping[None,:]]
        n = X.shape[0]
        X = X.reshape((n,m))
        Y = H[mapping[:,None],mask].reshape((m,n))

    scores = np.minimum(u_row[None,:],X).sum(axis=1) + np.minimum(u_col[:,None],Y).sum(axis=0)
    scores -= np.minimum(G[u,u],H.diagonal())

    return scores

In [4]:
m,n = 100,500
H_M,M_positions = get_highest_out_degrees(M,n)
H_F,F_positions = get_highest_out_degrees(F,m)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(m),np.arange(m)] = 0
H_M[np.arange(n),np.arange(n)] = 0
np.count_nonzero(H_M)/n**2, np.count_nonzero(H_F)/m**2

(0.204576, 0.2318)

In [ ]:
G=H_F#np.log1p(H_F)
H=H_M#np.log1p(H_M)
rng = np.random.default_rng()
result_matrix = np.zeros((m,n),dtype=int)

for i in range(200):
    mapping = rng.choice(n,size=m,replace=False)

    unmapped = np.ones(n,dtype=bool)
    unmapped[mapping] = False

    top_score = score(G,H,mapping,require_surjective=False)
    
    for j in range(50):

        unchanged = True

        for u in rng.permutation(m):

            change_scores = find_best_move(G,H,mapping,u)
            v = (change_scores*unmapped).argmax()

            if change_scores[v] > change_scores[mapping[u]]:
                top_score = top_score + change_scores[v]-change_scores[mapping[u]]

                unmapped[mapping[u]] = True
                unmapped[v] = False
                mapping[u] = v
                unchanged = False

            print(f"{i=}, {j=}, {top_score=}"+" "*20,end="\r")

        if unchanged:
            break
    
    result_matrix[np.arange(m),mapping] += 1


score(G,H,mapping,require_surjective=False), score(G,G,np.arange(m))

(7678, 13260)

In [ ]:
best_arg = result_matrix.argmax(axis=1)
best_scores = result_matrix[np.arange(m),best_arg]
np.concatenate([np.arange(m)[:,None],best_arg[:,None],best_scores[:,None]],axis=1)
#score(G,H,best_arg,require_surjective=False)

In [10]:
result_matrix[:,22],result_matrix[:,7]

(array([ 0,  0,  0, 15,  0,  0,  0,  0,  0,  4,  6,  0,  0,  3,  6,  1,  0,
         0,  0,  0,  0,  4, 20,  0,  0, 12,  2,  0,  0,  0,  0,  0,  0,  0,
         6,  0, 26,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0,
         0,  1,  1,  0,  1,  0, 25,  0,  0,  0,  0,  0,  0,  0,  4,  0,  0,
         0,  0,  0,  0, 18,  0,  0,  0,  5,  0,  0,  0,  2,  0,  0,  0,  0,
         0,  1,  1,  0,  1,  0,  0,  0,  1,  0,  1,  0,  0,  0,  0]),
 array([ 0,  1,  0, 12,  0,  1,  0,  0,  1,  4,  6,  0,  0,  4, 16,  0,  0,
         1,  0,  0,  0,  5, 14,  0,  0,  7,  3,  0,  0,  0,  1,  0,  0,  0,
         7,  0,  8,  1,  1,  0,  1,  0,  0,  3,  1,  0,  0,  1,  0,  1,  1,
         0,  3,  0,  1,  2,  1,  9,  1,  0,  0,  0,  0,  1,  2,  1,  1,  0,
         0,  0,  0,  1,  8,  0,  1,  0,  3,  0,  1,  4,  0,  1,  0,  0,  0,
         0,  0,  1,  1, 10,  1,  0,  0,  2,  0,  4,  0,  0,  0,  0]))

In [126]:
mapping = best_arg.copy()
top_score = score(G,H,best_arg,require_surjective=False)

for j in range(50):

    unchanged = True

    for u in rng.permutation(m):
        best = mapping[u]
        for v in range(n):

            if u==v:
                continue

            mapping[u] = v
            s = score(G,H,mapping,require_surjective=False)
            if s > top_score:
                top_score = s
                best = v
                unchanged = False
            else:
                mapping[u] = best

        print(f"{i=}, {j=}, {top_score=}",end="\r")

    if unchanged:
        break